In [ ]:
#Packages
import sys
import math
import matplotlib.pyplot as plt
import json
import csv
import numpy as np
import pandas as pd
import time
import re
import random
from tqdm import tqdm
import os
import multiprocessing as mp
import prince

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from scipy.stats import loguniform
from skopt import BayesSearchCV
from skopt.plots import plot_objective, plot_histogram
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import ShuffleSplit
from sklearn import linear_model

# Load and Prepare Data

In [ ]:
#Prepare USDA and Phenol Expore data
uspx_data=pd.read_csv('USPX_features.csv')
uspx_data=uspx_data[uspx_data['value']!=0] #Remove rows reporting 0 concentration
uspx_data=uspx_data.reset_index()
uspx_data=uspx_data.drop(columns=['index'])

#Split data
uspx_comp_ids=uspx_data[['row_id','comp_name','InChIKey']] #Chemical Identifiers
Slbl=uspx_data[['row_id','Cvariable','C_Label','F_Label']] #Labels for Stratifying Folds
uspx_props=uspx_data[['MW','C','logP','logS','HBI','NCA','NPSA','C_Label']] #Property Features
uspx_phylo=uspx_data[['class','order','family','genus','F_Label']] #Phylogenetic Features
uspx_resp=uspx_data[['row_id','logvalue']] #Response Values

# Find Principal Components for Compounds and Foods

In [ ]:
#Merge all chemical property lists of all datasets
uspx_props=uspx_props.drop_duplicates() #Remove duplicate compounds
uspx_props=uspx_props.reset_index(drop=True)
Clbl=pd.DataFrame(uspx_props['C_Label']) #Create Compound Label list of merged dataset
uspx_props=uspx_props.drop(columns=['C_Label']) #Drop label column for input into PCA

#Merge all phylogenetic lists of all datasets
uspx_phylo=uspx_phylo.drop_duplicates() #Remove dulplicate foods
uspx_phylo=uspx_phylo.reset_index(drop=True)
Flbl=pd.DataFrame(uspx_phylo['F_Label']) #Create Phylogenetic Label list of merged dataset
uspx_phylo=uspx_phylo.drop(columns=['F_Label']) #Drop label column for input into MCA

In [ ]:
#Find Principle Components of property features for merged dataset
pca=PCA(n_components=7)
pcc_df=pca.fit_transform(uspx_props) #Find principal components for the chemical properties
pcc_df=pd.DataFrame(data=pcc_df,columns=['PCC1','PCC2','PCC3','PCC4','PCC5','PCC6','PCC7'])
print('prop explained:',pca.explained_variance_ratio_)

#Find Components of phylogenetic features for merged dataset
mca=prince.MCA(n_components=13)
mca.fit(uspx_phylo)
pcp_df=mca.transform(uspx_phylo) #Find principal components for the phylogenetic tree
pcp_df.columns=['PCP1','PCP2','PCP3','PCP4','PCP5','PCP6','PCP7','PCP8','PCP9','PCP10','PCP11','PCP12','PCP13']
print('pt explained:',mca.total_inertia_)
print(sum(mca.explained_inertia_))

# Create Training Data out of Principal Components

In [ ]:
#Create a USDA Phenol Explorer Training Dataset from the PCA and MCA analysis components
n1=len(uspx_data)
xx=np.linspace(0,(n1*1),(n1*1)+1)
USPX=pd.DataFrame(columns=['PCC1','PCC2','PCC3','PCC4','PCP1','PCP2','PCP3','PCP4','PCP5','PCP6','PCP7','PCP8','PCP9','PCP10','PCP11','PCP12','PCP13'], index=xx)

for h in uspx_data.index:
    #Find property component features for each row in USPX data
    S1=Clbl.loc[Clbl['C_Label']==uspx_data['C_Label'][h]]
    USPX.loc[h,('PCC1','PCC2','PCC3','PCC4')]=[pcc_df.loc[S1.index[0],'PCC1'],pcc_df.loc[S1.index[0],'PCC2'],pcc_df.loc[S1.index[0],'PCC3'],pcc_df.loc[S1.index[0],'PCC4']]
    
    #Find phylogenetic component features for each row in USPX data
    S1=Flbl.loc[Flbl['F_Label']==uspx_data['F_Label'][h]]
    USPX.loc[h,('PCP1','PCP2','PCP3','PCP4','PCP5','PCP6','PCP7','PCP8','PCP9','PCP10','PCP11','PCP12','PCP13')]=[pcp_df.loc[S1.index[0],'PCP1'],pcp_df.loc[S1.index[0],'PCP2'],pcp_df.loc[S1.index[0],'PCP3'],pcp_df.loc[S1.index[0],'PCP4'],pcp_df.loc[S1.index[0],'PCP5'],pcp_df.loc[S1.index[0],'PCP6'],pcp_df.loc[S1.index[0],'PCP7'],pcp_df.loc[S1.index[0],'PCP8'],pcp_df.loc[S1.index[0],'PCP9'],pcp_df.loc[S1.index[0],'PCP10'],pcp_df.loc[S1.index[0],'PCP11'],pcp_df.loc[S1.index[0],'PCP12'],pcp_df.loc[S1.index[0],'PCP13']]
USPX=USPX.dropna()
USPX=USPX.reset_index(drop=True)

# Building Model

In [ ]:
#HyperParameter Optimization
X=USPX
Y=uspx_resp['logvalue']

#Stratified folds by Chemical Classes
y=Slbl['Cvariable']
skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
cvf=skf.split(X,y)

#Find optimization start time
t=time.time()

#Bayesian optimizer and parameters
opt=BayesSearchCV(
    estimator=GradientBoostingRegressor(),
    search_spaces={'max_depth':(3,10),
    'max_leaf_nodes':(2,50),
    'learning_rate':(0.01,1.0,'log-uniform'),
    'n_estimators':(10,200)},
    n_iter=100,
    verbose=0,
    cv=cvf
)
opt.fit(X,np.ravel(Y))

#Duration of optimization and best fit hyperparameters
print('Elapsed: %s' % (time.time()-t))
print("score: %s" % opt.best_score_)
print(opt.best_estimator_)

In [ ]:
#All Hyperparameter Optimization Results
pd.DataFrame(opt.cv_results_)

In [ ]:
np.mean(opt.cv_results_['mean_test_score'])

In [ ]:
d=pd.DataFrame(opt.cv_results_)
d.loc[d['mean_test_score']==opt.best_score_]

In [ ]:
mdl=GradientBoostingRegressor(
    n_estimators=opt.best_params_['n_estimators'],
    max_leaf_nodes=opt.best_params_['max_leaf_nodes'],
    learning_rate=opt.best_params_['learning_rate'],
    max_depth=opt.best_params_['max_depth'])

# Perform Leave One Out

In [ ]:
#Create Save File
n1=len(USPX)
xx=np.linspace(0,(n1*1),(n1*1)+1)
pairs=pd.DataFrame(columns=['row_id','real','pred','Fold','AFold','MSE'], index=xx)

r=0
t=time.time()
T=[]
for h in USPX.index:
    t1=time.time()
    pairs.row_id[r]=uspx_comp_ids['row_id'][h]
    
    # Select compound
    Test_X=USPX.iloc[[h]]
    Test_Y=uspx_resp.iloc[[h]]
    Test_Y=Test_Y['logvalue']
    
    # Drop selected compound from training 
    Train_X=USPX.drop(h)
    Train_Y=uspx_resp.drop(h)
    Train_Y=Train_Y['logvalue']
    
    # Predict concentration for select compound
    mdl.fit(Train_X,np.ravel(Train_Y))
    mse=mean_squared_error(np.ravel(Test_Y),mdl.predict(Test_X))
    prd=mdl.predict(Test_X)[0]
    rl=float(Test_Y[h])
    pairs.real[r]=rl
    pairs.pred[r]=prd
    pairs.MSE[r]=mse
    
    # Calculate fold change and absolute fold change
    if rl<prd:
        pairs.Fold[r]=np.exp(prd)/np.exp(rl)
        pairs.AFold[r]=np.exp(prd)/np.exp(rl)
    else:
        pairs.Fold[r]=np.exp(rl)/np.exp(prd)*-1
        pairs.AFold[r]=np.exp(rl)/np.exp(prd)
    r=r+1
    
    T.append(time.time()-t1)
    print('Percent Complete:',"{:.2%}".format(h/len(USPX.index)),'Estimated Time Remaining=', 'hrs:',round(((len(USPX.index)-h)*np.mean(T))/60/60,3), 'min:',round(((len(USPX.index)-h)*np.mean(T))/60,3),end="\r")
    
    #Saves a file after every 1000 for insurance
    if h>0:
        check=h/1000
        if (check-int(check)==0)==True:
            filename='USPX_iter_'+str(h)+'.csv'
            pairs.to_csv(filename,sep=',')
    
print('Elapsed: %s' % (time.time()-t))
pairs=pairs.dropna(axis=0)

In [ ]:
pairs.to_csv('USPX-predictions.csv')

# Calculate Fold Changes

In [ ]:
#Create Save File
n1=30
xx=np.linspace(0,(n1*1),(n1*1)+1)
FDP=pd.DataFrame(columns=['FD','Percent'], index=xx)

# Find percent of Fold Changes (FC) at each integer threshold of FC
for h in range(1,n1+1):
    cnt=0
    for j in pairs['AFold']:
        if j<h:
            cnt=cnt+1
    
    per=cnt/len(pairs)
    FDP.FD[h]=h
    FDP.Percent[h]=per
FDP=FDP.dropna(axis=0)

# Basic Stats on Leave one out

In [ ]:
print("Mean MSE: ",np.mean(pairs['MSE']))
print("std MSE: ",np.std(pairs['MSE']))
print("Median MSE: ",np.median(pairs['MSE']))
print("Mean FD: ",np.mean(pairs['AFold']))
print("std FD: ",np.std(pairs['AFold']))
print("Median FD: ",np.median(pairs['AFold']))